In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Open the URL
driver.get("https://web.afrcmp.org/feesinformation/frm_showinstitutes.aspx")

# Streams to process
streams = [
    {"name": "Higher Education", "id": "rblstream_0"},
    {"name": "Medical & Paramedical Education", "id": "rblstream_1"},
    {"name": "Technical Education", "id": "rblstream_2"},
    {"name": "University", "id": "rblstream_3"}
]

# Function to sanitize directory and file names
def sanitize_name(name):
    # Replace characters that are not safe in file names or paths
    return name.replace("&", "and").replace("/", "-").replace("\\", "-").replace(":", "-").replace("*", "").replace("?", "").replace("\"", "'").replace("<", "").replace(">", "").replace("|", "")

# Create a list to store all data frames
all_dfs = []

# Function to handle StaleElementReferenceException and retry
def retry_on_stale_element(func):
    def wrapper(*args, **kwargs):
        tries = 3
        while tries > 0:
            try:
                return func(*args, **kwargs)
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Retrying...")
                tries -= 1
                if tries == 0:
                    print("Maximum retries exceeded. Skipping...")
                    break
            except NoSuchElementException as e:
                print(f"NoSuchElementException occurred: {e}")
                raise e  # Raise the exception to handle externally if needed
    return wrapper

# Loop through each stream
for stream in streams:
    print(f"Processing data for stream: {stream['name']}")

    try:
        # Step 1: Check the respective stream
        @retry_on_stale_element
        def select_stream():
            stream_radio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, stream['id'])))
            stream_radio.click()

        select_stream()

        # Step 3: Select the year to be 2024-25
        @retry_on_stale_element
        def select_year():
            year_select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlYear"))))
            year_select.select_by_visible_text("2024-2025")

        select_year()

        # Step 4: Select Division to be --ALL--
        @retry_on_stale_element
        def select_division():
            division_select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "DdlDivision"))))
            division_select.select_by_value("0")

        select_division()

        # Step 5: Click on "Show"
        @retry_on_stale_element
        def click_show():
            show_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnShow")))
            show_button.click()

        click_show()

        # Wait for the table to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlProgram")))

        # Get the course options
        course_select = Select(driver.find_element(By.ID, "ddlProgram"))
        courses = course_select.options

        # Accumulate data for each stream and course in a list
        stream_data = []

        def click_pages(counter):
            if counter == 0:
                start = 1
            if counter == 1:
                start = 2
            for i in range(start,len(driver.find_element(By.CLASS_NAME, 'Pager').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td'))):
                print("firstelement ", driver.find_element(By.XPATH, '//*[@id="grdShowInstitutes"]/tbody/tr[2]/td[1]').text)
                driver.find_element(By.CLASS_NAME, 'Pager').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[i].click()
                time.sleep(2)
            check_further_pages(1)
        def click_remaining_pages():
            for i in range(1,len(driver.find_element(By.CLASS_NAME, 'Pager').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td'))):
                print("firstelement ", driver.find_element(By.XPATH, '//*[@id="grdShowInstitutes"]/tbody/tr[2]/td[1]').text)
                driver.find_element(By.CLASS_NAME, 'Pager').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[i].click()
                time.sleep(2)
        
            
        def check_further_pages(counter):
            if driver.find_element(By.CLASS_NAME, 'Pager').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'td')[-1].text ==  '...':
                click_pages(counter)
            else:
                click_remaining_pages()
        
        check_further_pages(0)


        # Function to retry selecting course and extracting data including pagination
        @retry_on_stale_element
        def retry_select_course(course_index):
            tries = 3
            while tries > 0:
                try:
                    # Re-select the course dropdown and get options again to avoid StaleElementReferenceException
                    course_select = Select(driver.find_element(By.ID, "ddlProgram"))
                    course_select.select_by_index(course_index)
                    course_name = course_select.first_selected_option.text  # Get the course name

                    # Extract the table data
                    stream_data = []  # List to store data for each page

                    # Function to extract data from current page
                    def extract_page_data():
                        rows = driver.find_elements(By.XPATH, '//*[@class="Item" or @class="AlternatingItem"]')
                        data = []
                        for row in rows:
                            cells = row.find_elements(By.TAG_NAME, 'td')
                            row_data = [cell.text.strip() for cell in cells]
                            row_data.append(course_name)  # Add the course name to each row
                            data.append(row_data)
                        return data

                    # Initial page extraction
                    page_data = extract_page_data()
                    stream_data.extend(page_data)

                    # Check for pagination
                    pager_row = driver.find_elements(By.XPATH, '//tr[@class="Pager"]')
                   
                    if pager_row:
                        pager_links = pager_row[0].find_elements(By.TAG_NAME, 'a')
                        num_pages = len(pager_links) + 1  # Include current page

                        # Iterate over remaining pages
                        for page_num in range(2, num_pages + 1):
                            try:
                                driver.execute_script("__doPostBack('grdShowInstitutes', 'Page$" + str(page_num) + "')")
                                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlProgram")))

                                # Extract data from the current page
                                page_data = extract_page_data()
                                stream_data.extend(page_data)

                            except TimeoutException:
                                print(f"TimeoutException occurred while loading page {page_num} for '{course_name}'. Skipping...")
                                continue

                    # Ensure the correct number of columns
                    headers = ['S.No', 'Name Of institute', 'Address', 'Place', 'Year wise Fees (in Rs)', 'Course Name']
                    max_columns = len(headers)
                    stream_data = [row[:max_columns] for row in stream_data]  # Truncate extra columns if any

                    # Create a DataFrame from the extracted data
                    df = pd.DataFrame(stream_data, columns=headers)
                    df['Course Name'] = course_name  # Fill the Course Name column

                    print(f"Data for course '{course_name}' in stream '{stream['name']}' has been extracted.")
                    return df  # Return DataFrame if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    tries -= 1
                    if tries == 0:
                        print("Maximum retries exceeded. Skipping course selection.")
                        break
                except NoSuchElementException as e:
                    print(f"NoSuchElementException occurred: {e}")
                    raise e  # Raise the exception to handle externally if needed

        # Step 6: Extract the table data for each course
        for course_index in range(1, len(courses)):  # Start from index 1 to skip the default selection
            df = retry_select_course(course_index)
            if df is not None:
                stream_data.append(df)  # Append DataFrame to stream data list

        # Concatenate all DataFrames for the current stream
        if stream_data:
            stream_df = pd.concat(stream_data, ignore_index=True)
            all_dfs.append(stream_df)

    except TimeoutException:
        print(f"TimeoutException occurred while processing stream: {stream['name']}. Skipping...")

# Close the browser
driver.quit()

# Combine all DataFrames into a single DataFrame
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Sanitize stream names for file naming
    stream_names = [sanitize_name(stream['name']) for stream in streams]
    stream_file_name = '_'.join(stream_names)

    # Save the combined DataFrame to an Excel file
    output_file_path = os.path.join("output_excel_files", f'institutes_and_fees_information_{stream_file_name}.xlsx')
    combined_df.to_excel(output_file_path, index=False)

    print("All data has been extracted and saved to:", output_file_path)
else:
    print("No data was extracted. Check the script execution.")

#here the next page is opening but it is giving error

Processing data for stream: Higher Education
StaleElementReferenceException occurred. Retrying...


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79A96EEB2+31554]
	(No symbol) [0x00007FF79A8E7EE9]
	(No symbol) [0x00007FF79A7A872A]
	(No symbol) [0x00007FF79A7AE55E]
	(No symbol) [0x00007FF79A7B0EB1]
	(No symbol) [0x00007FF79A7B0F50]
	(No symbol) [0x00007FF79A7F4137]
	(No symbol) [0x00007FF79A81D02A]
	(No symbol) [0x00007FF79A7EBA76]
	(No symbol) [0x00007FF79A81D240]
	(No symbol) [0x00007FF79A83C977]
	(No symbol) [0x00007FF79A81CDD3]
	(No symbol) [0x00007FF79A7EA33B]
	(No symbol) [0x00007FF79A7EAED1]
	GetHandleVerifier [0x00007FF79AC78B2D+3217341]
	GetHandleVerifier [0x00007FF79ACC5AF3+3532675]
	GetHandleVerifier [0x00007FF79ACBB0F0+3489152]
	GetHandleVerifier [0x00007FF79AA1E786+750614]
	(No

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
import time

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Open the URL
driver.get("https://web.afrcmp.org/feesinformation/frm_showinstitutes.aspx")

# Streams to process
streams = [
    {"name": "Higher Education", "id": "rblstream_0"},
    {"name": "Medical & Paramedical Education", "id": "rblstream_1"},
    {"name": "Technical Education", "id": "rblstream_2"},
    {"name": "University", "id": "rblstream_3"}
]

# Function to sanitize directory and file names
def sanitize_name(name):
    # Replace characters that are not safe in file names or paths
    return name.replace("&", "and").replace("/", "-").replace("\\", "-").replace(":", "-").replace("*", "").replace("?", "").replace("\"", "'").replace("<", "").replace(">", "").replace("|", "")

# Create a list to store all data frames
all_dfs = []

# Function to handle StaleElementReferenceException and retry
def retry_on_stale_element(func):
    def wrapper(*args, **kwargs):
        tries = 3
        while tries > 0:
            try:
                return func(*args, **kwargs)
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Retrying...")
                tries -= 1
                if tries == 0:
                    print("Maximum retries exceeded. Skipping...")
                    break
            except NoSuchElementException as e:
                print(f"NoSuchElementException occurred: {e}")
                raise e  # Raise the exception to handle externally if needed
    return wrapper

# Loop through each stream
for stream in streams:
    print(f"Processing data for stream: {stream['name']}")

    try:
        # Step 1: Check the respective stream
        @retry_on_stale_element
        def select_stream():
            stream_radio = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, stream['id'])))
            stream_radio.click()

        select_stream()

        # Step 3: Select the year to be 2024-25
        @retry_on_stale_element
        def select_year():
            year_select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlYear"))))
            year_select.select_by_visible_text("2024-2025")

        select_year()

        # Step 4: Select Division to be --ALL--
        @retry_on_stale_element
        def select_division():
            division_select = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "DdlDivision"))))
            division_select.select_by_value("0")

        select_division()

        # Step 5: Click on "Show"
        @retry_on_stale_element
        def click_show():
            show_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "btnShow")))
            show_button.click()

        click_show()

        # Wait for the table to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlProgram")))

        # Get the course options
        course_select = Select(driver.find_element(By.ID, "ddlProgram"))
        courses = course_select.options

        # Accumulate data for each stream and course in a list
        stream_data = []

        # Function to retry selecting course and extracting data including pagination
        @retry_on_stale_element
        def retry_select_course(course_index):
            tries = 3
            while tries > 0:
                try:
                    # Re-select the course dropdown and get options again to avoid StaleElementReferenceException
                    course_select = Select(driver.find_element(By.ID, "ddlProgram"))
                    course_select.select_by_index(course_index)
                    course_name = course_select.first_selected_option.text  # Get the course name

                    # Extract the table data
                    data = []

                    # Function to extract data from current page
                    def extract_page_data():
                        rows = driver.find_elements(By.XPATH, '//*[@class="Item" or @class="AlternatingItem"]')
                        page_data = []
                        for row in rows:
                            cells = row.find_elements(By.TAG_NAME, 'td')
                            row_data = [cell.text.strip() for cell in cells]
                            row_data.append(course_name)  # Add the course name to each row
                            page_data.append(row_data)
                        return page_data

                    # Initial page extraction
                    page_data =extract_page_data()
                    data.extend(page_data)

                    # Check for pagination
                    pager_row = driver.find_elements(By.XPATH, '//tr[@class="Pager"]')
                    if pager_row:
                        pager_links = pager_row[0].find_elements(By.TAG_NAME, 'a')
                        num_pages = len(pager_links) + 1  # Include current page

                        # Iterate over remaining pages
                        for page_num in range(2, num_pages + 1):
                            try:
                                driver.execute_script("__doPostBack('grdShowInstitutes', 'Page$" + str(page_num) + "')")
                                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlProgram")))

                                # Extract data from the current page
                                page_data = extract_page_data()
                                data.extend(page_data)

                            except TimeoutException:
                                print(f"TimeoutException occurred while loading page {page_num} for '{course_name}'. Skipping...")
                                continue

                    # Ensure the correct number of columns
                    headers = ['S.No', 'Name Of institute', 'Address', 'Place', 'Year wise Fees (in Rs)', 'Course Name']
                    max_columns = len(headers)
                    data = [row[:max_columns] for row in data]  # Truncate extra columns if any

                    # Create a DataFrame from the extracted data
                    df = pd.DataFrame(data, columns=headers)
                    df['Course Name'] = course_name  # Fill the Course Name column

                    print(f"Data for course '{course_name}' in stream '{stream['name']}' has been extracted.")
                    return df  # Return DataFrame if successful

                except StaleElementReferenceException:
                    print("StaleElementReferenceException occurred. Retrying...")
                    tries -= 1
                    if tries == 0:
                        print("Maximum retries exceeded. Skipping course selection.")
                        break
                except NoSuchElementException as e:
                    print(f"NoSuchElementException occurred: {e}")
                    raise e  # Raise the exception to handle externally if needed

        # Step 6: Extract the table data for each course
        for course_index in range(1, len(courses)):  # Start from index 1 to skip the default selection
            df = retry_select_course(course_index)
            if df is not None:
                stream_data.append(df)  # Append DataFrame to stream data list

        # Concatenate all DataFrames for the current stream
        if stream_data:
            stream_df = pd.concat(stream_data, ignore_index=True)
            all_dfs.append(stream_df)

    except TimeoutException:
        print(f"TimeoutException occurred while processing stream: {stream['name']}. Skipping...")

# Close the browser
driver.quit()

# Combine all DataFrames into a single DataFrame
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Sanitize stream names for file naming
    stream_names = [sanitize_name(stream['name']) for stream in streams]
    stream_file_name = '_'.join(stream_names)

    # Save the combined DataFrame to an Excel file
    output_file_path = os.path.join("output_excel_files", f'institutes_and_fees_information_{stream_file_name}.xlsx')
    combined_df.to_excel(output_file_path, index=False)

    print("All data has been extracted and saved to:", output_file_path)
else:
    print("No data was extracted. Check the script execution.")

Processing data for stream: Higher Education
StaleElementReferenceException occurred. Retrying...
StaleElementReferenceException occurred. Retrying...
Data for course 'M.P.ED.' in stream 'Higher Education' has been extracted.
StaleElementReferenceException occurred. Retrying...
StaleElementReferenceException occurred. Retrying...
StaleElementReferenceException occurred. Retrying...
Maximum retries exceeded. Skipping course selection.
NoSuchElementException occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ddlProgram"]"}
  (Session info: chrome=126.0.6478.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79A96EEB2+31554]
	(No symbol) [0x00007FF79A8E7EE9]
	(No symbol) [0x00007FF79A7A872A]
	(No symbol) [0x00007FF79A7F8434]
	(No symbol) [0x00007FF79A7F853C]
	(No symbol) [0x00007FF79A83F6A7]
	(No s

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ddlProgram"]"}
  (Session info: chrome=126.0.6478.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79A96EEB2+31554]
	(No symbol) [0x00007FF79A8E7EE9]
	(No symbol) [0x00007FF79A7A872A]
	(No symbol) [0x00007FF79A7F8434]
	(No symbol) [0x00007FF79A7F853C]
	(No symbol) [0x00007FF79A83F6A7]
	(No symbol) [0x00007FF79A81D06F]
	(No symbol) [0x00007FF79A83C977]
	(No symbol) [0x00007FF79A81CDD3]
	(No symbol) [0x00007FF79A7EA33B]
	(No symbol) [0x00007FF79A7EAED1]
	GetHandleVerifier [0x00007FF79AC78B2D+3217341]
	GetHandleVerifier [0x00007FF79ACC5AF3+3532675]
	GetHandleVerifier [0x00007FF79ACBB0F0+3489152]
	GetHandleVerifier [0x00007FF79AA1E786+750614]
	(No symbol) [0x00007FF79A8F376F]
	(No symbol) [0x00007FF79A8EEB24]
	(No symbol) [0x00007FF7